In [ ]:
import pandas as pd
import pandas_ta
import requests
import numpy as np
import yfinance as yf
import datetime as dt

In [13]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

sp500 = pd.read_html(url)[0]

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2023-09-27'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[**********************65%%*****                 ]  327 of 503 completed

$BF.B: possibly delisted; No price data found  (1d 2015-09-29 00:00:00 -> 2023-09-27)


[*********************100%%**********************]  503 of 503 completed

6 Failed downloads:
['SW', 'SOLV', 'GEV']: YFChartError("%ticker%: Data doesn't exist for startDate = 1443499200, endDate = 1695787200")
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2015-09-29 00:00:00 -> 2023-09-27)')
['VLTO']: YFChartError('%ticker%: No data found, symbol may be delisted')
/var/folders/zy/64gtqsbx4ts83dlyf7nwkjg00000gn/T/ipykernel_20845/1606964230.py:13: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  end=end_date).stack()


Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.425232   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361614   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.622629   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.004154   52.790001   54.189999   51.880001   53.099998   
           ABT      33.302017   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.501099   89.519997   90.849998   89.500000   90.379997   
           YUM     121.604256  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.131134  176.869995  178.449997  176.270004  176.580002   

Price                   volume  
date       ticker               
2015-09-29 A         2252400.0  
           AAL       7478800.0  
           AAPL    293461600.0  
           ABBV     12842800.0  
           ABT      12287500.0  
...                        ...  
2023-09-26 XYL       1322400.0  
           YUM       1500600.0  
           ZBH       3610500.0  
           ZBRA       355400.0  
           ZTS       1463200.0  

[981931 rows x 6 columns]

In [23]:
"""
Calculate featrues such as Dollar Volume and rsi
Dollar volume -> # of shares traded multiplied by the price, used to measure cash flow on D2D. 
rsi -> Relative Strength Index, used to measure a stocks momentum.
"""

# df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))
df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df


Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-10-14 AAL      41.767193   43.799999   44.500000   43.230000   44.000000   
           AAPL     24.882265   27.552500   27.879999   27.389999   27.822500   
           ABBV     37.128017   53.939999   54.619999   53.570000   54.090000   
           ABT      34.529949   40.720001   41.340000   40.660000   40.970001   
           ACN      89.066177  102.430000  102.949997  101.199997  102.220001   
...                       ...         ...         ...         ...         ...   
2023-09-26 VRTX    351.690002  351.690002  352.540009  349.170013  350.000000   
           VZ       30.755375   32.990002   33.169998   32.810001   32.889999   
           WFC      39.479488   40.650002   41.310001   40.360001   41.049999   
           WMT      53.445198   54.166668   54.513332   54.116669   54.313332   
           XOM     112.466652  116.410004  116.730003  115.370003  115.500000   

Price                   volume  dollar_volume  
date       ticker                              
2015-10-14 AAL      11125800.0     464.693434  
           AAPL    177849600.0    4425.300894  
           ABBV      8727400.0     324.031059  
           ABT       6417200.0     221.585590  
           ACN       2421100.0     215.638122  
...                        ...            ...  
2023-09-26 VRTX       633600.0     222.830786  
           VZ       18841600.0     579.480472  
           WFC      15219400.0     600.854125  
           WMT      14435700.0     771.518846  
           XOM      11805400.0    1327.713813  

[298149 rows x 7 columns]

In [ ]:
"""
Filter the 150 most liquid stocks
"""
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('ME').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('ME').last().stack('ticker')],
                  axis=1)).dropna()

data



In [20]:
"""
Calculate the 5 year rolling average
"""

df['dollar_volume'] = (df.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

df['dollar_vol_rank'] = (df.groupby('date')['dollar_volume'].rank(ascending=False))

df = df[df['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-10-14 AAL      41.767193   43.799999   44.500000   43.230000   44.000000   
           AAPL     24.882265   27.552500   27.879999   27.389999   27.822500   
           ABBV     37.128017   53.939999   54.619999   53.570000   54.090000   
           ABT      34.529949   40.720001   41.340000   40.660000   40.970001   
           ACN      89.066177  102.430000  102.949997  101.199997  102.220001   
...                       ...         ...         ...         ...         ...   
2023-09-26 VRTX    351.690002  351.690002  352.540009  349.170013  350.000000   
           VZ       30.755375   32.990002   33.169998   32.810001   32.889999   
           WFC      39.479488   40.650002   41.310001   40.360001   41.049999   
           WMT      53.445198   54.166668   54.513332   54.116669   54.313332   
           XOM     112.466652  116.410004  116.730003  115.370003  115.500000   

Price                   volume  
date       ticker               
2015-10-14 AAL      11125800.0  
           AAPL    177849600.0  
           ABBV      8727400.0  
           ABT       6417200.0  
           ACN       2421100.0  
...                        ...  
2023-09-26 VRTX       633600.0  
           VZ       18841600.0  
           WFC      15219400.0  
           WMT      14435700.0  
           XOM      11805400.0  

[298149 rows x 6 columns]

In [36]:
"""
Calculate monthly returns for different time horizons
"""

def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df
    
    
data = df.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-10-30 AAL      44.074894   46.220001   46.770000   45.709999   45.770000   
           AAPL     26.979687   29.875000   30.305000   29.862499   30.247499   
           ABBV     40.989502   59.549999   60.730000   57.660000   57.869999   
           ABT      37.989731   44.799999   45.340000   44.759998   45.189999   
           ACN      93.213814  107.199997  108.389999  107.199997  108.050003   
...                       ...         ...         ...         ...         ...   
2023-09-26 VRTX    351.690002  351.690002  352.540009  349.170013  350.000000   
           VZ       30.755375   32.990002   33.169998   32.810001   32.889999   
           WFC      39.479488   40.650002   41.310001   40.360001   41.049999   
           WMT      53.445198   54.166668   54.513332   54.116669   54.313332   
           XOM     112.466652  116.410004  116.730003  115.370003  115.500000   

Price                   volume  dollar_volume  return_1m  return_2m  \
date       ticker                                                     
2015-10-30 AAL       8589700.0     378.590117   0.007850   0.003917   
           AAPL    197461200.0    5327.441319  -0.008546   0.000964   
           ABBV     27465600.0    1125.801265   0.058624   0.042486   
           ABT       8442300.0     320.720705  -0.009726   0.000447   
           ACN       2529200.0     235.756378  -0.006948  -0.011596   
...                        ...            ...        ...        ...   
2023-09-26 VRTX       633600.0     222.830786   0.002909   0.003085   
           VZ       18841600.0     579.480472  -0.005127  -0.004366   
           WFC      15219400.0     600.854125  -0.021896  -0.007059   
           WMT      14435700.0     771.518846  -0.003495   0.000462   
           XOM      11805400.0    1327.713813   0.001549   0.006374   

Price              return_3m  return_6m  return_9m  return_12m  
date       ticker                                               
2015-10-30 AAL     -0.001725   0.000832   0.004348    0.004492  
           AAPL     0.014202   0.005690   0.007498    0.006767  
           ABBV     0.034769   0.026349   0.006534    0.008279  
           ABT      0.006940   0.004920   0.006798    0.007989  
           ACN     -0.002075   0.001155   0.004342    0.003800  
...                      ...        ...        ...         ...  
2023-09-26 VRTX    -0.002443  -0.000147   0.001544    0.002075  
           VZ      -0.003013  -0.002702  -0.002888   -0.001153  
           WFC     -0.013331  -0.010965  -0.004280   -0.000714  
           WMT      0.001193  -0.000940  -0.001399   -0.000649  
           XOM      0.004770  -0.001750  -0.000029    0.000575  

[294819 rows x 13 columns]

In [37]:
"""
Filter out stocks wtih less than 10mo of data
"""

obsv = data.groupby(level=1).size()

good_stocks = obsv[obsv >= 10]

data = data[data.index.get_level_values('ticker').isin(good_stocks.index)]

data

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-10-30 AAL      44.074894   46.220001   46.770000   45.709999   45.770000   
           AAPL     26.979687   29.875000   30.305000   29.862499   30.247499   
           ABBV     40.989502   59.549999   60.730000   57.660000   57.869999   
           ABT      37.989731   44.799999   45.340000   44.759998   45.189999   
           ACN      93.213814  107.199997  108.389999  107.199997  108.050003   
...                       ...         ...         ...         ...         ...   
2023-09-26 VRTX    351.690002  351.690002  352.540009  349.170013  350.000000   
           VZ       30.755375   32.990002   33.169998   32.810001   32.889999   
           WFC      39.479488   40.650002   41.310001   40.360001   41.049999   
           WMT      53.445198   54.166668   54.513332   54.116669   54.313332   
           XOM     112.466652  116.410004  116.730003  115.370003  115.500000   

Price                   volume  dollar_volume  return_1m  return_2m  \
date       ticker                                                     
2015-10-30 AAL       8589700.0     378.590117   0.007850   0.003917   
           AAPL    197461200.0    5327.441319  -0.008546   0.000964   
           ABBV     27465600.0    1125.801265   0.058624   0.042486   
           ABT       8442300.0     320.720705  -0.009726   0.000447   
           ACN       2529200.0     235.756378  -0.006948  -0.011596   
...                        ...            ...        ...        ...   
2023-09-26 VRTX       633600.0     222.830786   0.002909   0.003085   
           VZ       18841600.0     579.480472  -0.005127  -0.004366   
           WFC      15219400.0     600.854125  -0.021896  -0.007059   
           WMT      14435700.0     771.518846  -0.003495   0.000462   
           XOM      11805400.0    1327.713813   0.001549   0.006374   

Price              return_3m  return_6m  return_9m  return_12m  
date       ticker                                               
2015-10-30 AAL     -0.001725   0.000832   0.004348    0.004492  
           AAPL     0.014202   0.005690   0.007498    0.006767  
           ABBV     0.034769   0.026349   0.006534    0.008279  
           ABT      0.006940   0.004920   0.006798    0.007989  
           ACN     -0.002075   0.001155   0.004342    0.003800  
...                      ...        ...        ...         ...  
2023-09-26 VRTX    -0.002443  -0.000147   0.001544    0.002075  
           VZ      -0.003013  -0.002702  -0.002888   -0.001153  
           WFC     -0.013331  -0.010965  -0.004280   -0.000714  
           WMT      0.001193  -0.000940  -0.001399   -0.000649  
           XOM      0.004770  -0.001750  -0.000029    0.000575  

[294802 rows x 13 columns]

In [38]:
"""
Clean data and reformat
"""
data = data.drop(columns='adj close', axis=1)
data = data.dropna()

data



Price                   close        high         low        open  \
date       ticker                                                   
2015-10-30 AAL      46.220001   46.770000   45.709999   45.770000   
           AAPL     29.875000   30.305000   29.862499   30.247499   
           ABBV     59.549999   60.730000   57.660000   57.869999   
           ABT      44.799999   45.340000   44.759998   45.189999   
           ACN     107.199997  108.389999  107.199997  108.050003   
...                       ...         ...         ...         ...   
2023-09-26 VRTX    351.690002  352.540009  349.170013  350.000000   
           VZ       32.990002   33.169998   32.810001   32.889999   
           WFC      40.650002   41.310001   40.360001   41.049999   
           WMT      54.166668   54.513332   54.116669   54.313332   
           XOM     116.410004  116.730003  115.370003  115.500000   

Price                   volume  dollar_volume  return_1m  return_2m  \
date       ticker                                                     
2015-10-30 AAL       8589700.0     378.590117   0.007850   0.003917   
           AAPL    197461200.0    5327.441319  -0.008546   0.000964   
           ABBV     27465600.0    1125.801265   0.058624   0.042486   
           ABT       8442300.0     320.720705  -0.009726   0.000447   
           ACN       2529200.0     235.756378  -0.006948  -0.011596   
...                        ...            ...        ...        ...   
2023-09-26 VRTX       633600.0     222.830786   0.002909   0.003085   
           VZ       18841600.0     579.480472  -0.005127  -0.004366   
           WFC      15219400.0     600.854125  -0.021896  -0.007059   
           WMT      14435700.0     771.518846  -0.003495   0.000462   
           XOM      11805400.0    1327.713813   0.001549   0.006374   

Price              return_3m  return_6m  return_9m  return_12m  
date       ticker                                               
2015-10-30 AAL     -0.001725   0.000832   0.004348    0.004492  
           AAPL     0.014202   0.005690   0.007498    0.006767  
           ABBV     0.034769   0.026349   0.006534    0.008279  
           ABT      0.006940   0.004920   0.006798    0.007989  
           ACN     -0.002075   0.001155   0.004342    0.003800  
...                      ...        ...        ...         ...  
2023-09-26 VRTX    -0.002443  -0.000147   0.001544    0.002075  
           VZ      -0.003013  -0.002702  -0.002888   -0.001153  
           WFC     -0.013331  -0.010965  -0.004280   -0.000714  
           WMT      0.001193  -0.000940  -0.001399   -0.000649  
           XOM      0.004770  -0.001750  -0.000029    0.000575  

[294802 rows x 12 columns]

In [ ]:
from sklearn.cluster import KMeans

data = data.drop('cluster', axis=1)

def get_clusters(df):
    df['cluster'] = KMeans(n_clusters=4,
                           random_state=0,
                           init=initial_centroids).fit(df).labels_
    return df

data = data.dropna().groupby('date', group_keys=False).apply(get_clusters)

data

In [40]:
def plot_clusters(data):

    cluster_0 = data[data['cluster']==0]
    cluster_1 = data[data['cluster']==1]
    cluster_2 = data[data['cluster']==2]
    cluster_3 = data[data['cluster']==3]

    plt.scatter(cluster_0.iloc[:,0] , cluster_0.iloc[:,6] , color = 'red', label='cluster 0')
    plt.scatter(cluster_1.iloc[:,0] , cluster_1.iloc[:,6] , color = 'green', label='cluster 1')
    plt.scatter(cluster_2.iloc[:,0] , cluster_2.iloc[:,6] , color = 'blue', label='cluster 2')
    plt.scatter(cluster_3.iloc[:,0] , cluster_3.iloc[:,6] , color = 'black', label='cluster 3')
    
    plt.legend()
    plt.show()
    return

In [ ]:
plt.style.use('ggplot')

for i in data.index.get_level_values('date').unique().tolist():
    
    g = data.xs(i, level=0)
    
    plt.title(f'Date {i}')
    
    plot_clusters(g)

In [ ]:
"""
Define a portfolio optimization function
"""
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

def optimize_weights(prices, lower_bound=0):
    
    returns = expected_returns.mean_historical_return(prices=prices,
                                                      frequency=252)
    
    cov = risk_models.sample_cov(prices=prices,
                                 frequency=252)
    
    ef = EfficientFrontier(expected_returns=returns,
                           cov_matrix=cov,
                           weight_bounds=(lower_bound, .1),
                           solver='SCS')
    
    weights = ef.max_sharpe()
    
    return ef.clean_weights()

In [45]:
"""
Short list of fresh stocks, to 
"""
stocks = data.index.get_level_values('ticker').unique().tolist()

new_df = yf.download(tickers=stocks,
                     start=data.index.get_level_values('date').unique()[0]-pd.DateOffset(months=12),
                     end=data.index.get_level_values('date').unique()[-1])

new_df

[*********************100%%**********************]  270 of 270 completed


Price       Adj Close                                                 \
Ticker            AAL        AAPL        ABBV        ABNB        ABT   
Date                                                                   
2014-10-30  38.184391   23.747076   40.772171         NaN  36.112827   
2014-10-31  39.179264   23.973499   42.291645         NaN  36.195847   
2014-11-03  39.795155   24.284260   42.151684         NaN  35.946739   
2014-11-04  40.458393   24.106684   41.758492         NaN  36.212463   
2014-11-05  40.032032   24.164398   41.678520         NaN  36.253971   
...               ...         ...         ...         ...        ...   
2023-09-19  13.190000  178.158340  147.780304  141.850006  98.619568   
2023-09-20  13.060000  174.596588  147.895844  138.009995  98.237503   
2023-09-21  13.060000  173.044495  147.722519  132.750000  96.915009   
2023-09-22  12.900000  173.900131  147.058212  132.199997  96.150894   
2023-09-25  12.910000  175.183563  148.897141  134.139999  95.484741   

Price                                                                   ...  \
Ticker             ACN        ADBE         ADI         ADP        ADSK  ...   
Date                                                                    ...   
2014-10-30   68.622543   68.570000   38.058174   63.485935   57.369999  ...   
2014-10-31   69.048141   70.120003   40.179710   65.761612   57.540001  ...   
2014-11-03   68.920448   69.910004   40.260689   65.729424   57.770000  ...   
2014-11-04   69.073662   71.070000   40.042061   66.863281   58.240002  ...   
2014-11-05   70.205765   71.370003   40.511707   67.788040   58.320000  ...   
...                ...         ...         ...         ...         ...  ...   
2023-09-19  312.285492  541.690002  175.684769  240.348083  212.059998  ...   
2023-09-20  311.763947  535.780029  173.060562  238.422089  207.729996  ...   
2023-09-21  307.040253  513.880005  170.858963  233.387146  204.059998  ...   
2023-09-22  311.065247  512.900024  172.470840  234.003067  204.039993  ...   
2023-09-25  311.950897  511.600006  172.421692  234.658112  205.669998  ...   

Price        Volume                                                          \
Ticker          WDC       WFC      WMB       WMT     WTW     WYNN       XOM   
Date                                                                          
2014-10-30  2081100  14107600  8034100  14617200  308115  1176100   9767900   
2014-10-31  2665900  21126300  6104000  22712100  479916  1527800  17493700   
2014-11-03  2785100  14349500  5049900  20683200  504566  1222900  13410000   
2014-11-04  2100100  14931500  5849100  20714700  440467  2088000  13498500   
2014-11-05  1223000  14496900  3944600  17021400  309965  2938200  12480000   
...             ...       ...      ...       ...     ...      ...       ...   
2023-09-19  3159000  15318100  5109500  14019600  795300  2036900  12562900   
2023-09-20  8476500  16788100  4624500  10120800  670700  1061300  11879900   
2023-09-21  3854700  14990500  5504300  18782400  620600  2358400  14400600   
2023-09-22  5078400  16780300  5599700  13215000  651100  2611300  13197300   
2023-09-25  2685900  10624000  4871000  10459500  477500  1387800  11316000   

Price                                   
Ticker          YUM      ZBH       ZTS  
Date                                    
2014-10-30  2917483   811022   2137900  
2014-10-31  8379384  1530786   2589500  
2014-11-03  4277047   813082   4071300  
2014-11-04  3248402  1050703   9723700  
2014-11-05  2750842  1231983  11878900  
...             ...      ...       ...  
2023-09-19  1299300   970200   1373600  
2023-09-20  1437200  1055900   1485500  
2023-09-21  1492700  1489200   1691800  
2023-09-22  1534100  1267300   1281800  
2023-09-25  1048500   886700   1551100  

[2240 rows x 1620 columns]

In [ ]:
""" 
Calculate the daily returns for each stock that cound end up in the portfolio.
"""
returns_dataframe = np.log(new_df['Adj Close']).diff()

portfolio_df = pd.DataFrame()

for start_date in fixed_dates.keys():
    
    try:

        end_date = (pd.to_datetime(start_date)+pd.offsets.MonthEnd(0)).strftime('%Y-%m-%d')

        cols = fixed_dates[start_date]

        optimization_start_date = (pd.to_datetime(start_date)-pd.DateOffset(months=12)).strftime('%Y-%m-%d')

        optimization_end_date = (pd.to_datetime(start_date)-pd.DateOffset(days=1)).strftime('%Y-%m-%d')
        
        optimization_df = new_df[optimization_start_date:optimization_end_date]['Adj Close'][cols]
        
        success = False
        try:
            weights = optimize_weights(prices=optimization_df,
                                   lower_bound=round(1/(len(optimization_df.columns)*2),3))

            weights = pd.DataFrame(weights, index=pd.Series(0))
            
            success = True
        except:
            print(f'Max Sharpe Optimization failed for {start_date}, Continuing with Equal-Weights')
        
        if success==False:
            weights = pd.DataFrame([1/len(optimization_df.columns) for i in range(len(optimization_df.columns))],
                                     index=optimization_df.columns.tolist(),
                                     columns=pd.Series(0)).T
        
        temp_df = returns_dataframe[start_date:end_date]

        temp_df = temp_df.stack().to_frame('return').reset_index(level=0)\
                   .merge(weights.stack().to_frame('weight').reset_index(level=0, drop=True),
                          left_index=True,
                          right_index=True)\
                   .reset_index().set_index(['Date', 'index']).unstack().stack()

        temp_df.index.names = ['date', 'ticker']

        temp_df['weighted_return'] = temp_df['return']*temp_df['weight']

        temp_df = temp_df.groupby(level=0)['weighted_return'].sum().to_frame('Strategy Return')

        portfolio_df = pd.concat([portfolio_df, temp_df], axis=0)
    
    except Exception as e:
        print(e)

portfolio_df = portfolio_df.drop_duplicates()

portfolio_df

In [ ]:
"""
Visualize Portoflio returns and compare to sp500 returns
"""
spy = yf.download(tickers="SPY", start='2015-01-01',end=dt.date.today())

spy_ret = np.log(spy[['Adj Close']]).diff().dropna().rename({'Adj Close' : 'SPY Buy&hold'}, axis=1)

portfolio_df = portfolio_df.merge(spt_ret, left_index=True, right_index=True)

portfolio_df

In [ ]:
import matplotlib.ticker as mtick

plt.style.use('ggplot')

portfolio_cumulative_return = np.exp(np.log1p(portfolio_df).cumsum())-1

portfolio_cumulative_return[:'2023-09-29'].plot(figsize=(16,6))

plt.title('Trading Strategies')

plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1))

plt.ylabel('Return')

# show plot
plt.show()